# Python datatable Intro

Checkout the [documentation](https://datatable.readthedocs.io/en/latest/index.html) for additional details. [This kaggle kernel](https://www.kaggle.com/sudalairajkumar/getting-started-with-python-datatable) also provides a nice walk-through.

In [3]:
import numpy as np
import pandas as pd
import datatable as dt

# Create a dataframe

In [72]:
dt.Frame({"n": [1, 3], "s": ["foo", "bar"]})

,n,s
,▪▪▪▪,▪▪▪▪
0,1,foo
1,3,bar


# Loading Data

Just as with R, `fread` can be used to read in the data file.

In [40]:
link_loan_07 = "https://raw.githubusercontent.com/dosei1/Lending-Club-Loan-Data/master/loans_2007.csv"

df = dt.fread(link_loan_07)

In [41]:
type(df)

datatable.Frame

The `datatable` package allows you to convert the data frame to a Pandas dataframe or even NumPy using the `to_pandas` and `to_numpy` methods, respectively.

In [42]:
numpy_df = df.to_numpy()
pandas_df = df.to_pandas()

print( type(numpy_df) )
print( type(pandas_df) )

<class 'numpy.ma.core.MaskedArray'>
<class 'pandas.core.frame.DataFrame'>


# Basic Properties

- Shape
- Column names
- Data types

In [43]:
print(df.shape)       # (nrows, ncols)
print(df.names[:5])   # top 5 column names
print(df.stypes[:5])  # column types(top 5)

(39252, 41)
('loan_amnt', 'int_rate', 'installment', 'emp_length', 'annual_inc')
(stype.float64, stype.float64, stype.float64, stype.int32, stype.float64)


In [55]:
dt.Frame(col_names = df.names, types = df.stypes)

,col_names,types
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,loan_amnt,stype.float64
1,int_rate,stype.float64
2,installment,stype.float64
3,emp_length,stype.int32
4,annual_inc,stype.float64
5,loan_status,stype.bool8
6,dti,stype.float64
7,delinq_2yrs,stype.float64
8,fico_range_high,stype.float64


In [44]:
df.head(10)

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,zip_code,dti,delinq_2yrs,fico_range_high,…,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,5000,10.65,162.87,10,24000,1,860,27.65,0,739,…,0,0,0,1,0
1,2500,15.27,59.83,0,30000,0,309,1,0,744,…,0,0,0,0,1
2,2400,15.96,84.33,10,12252,1,606,8.72,0,739,…,1,0,0,1,0
3,10000,13.49,339.31,10,49200,1,917,20,0,694,…,0,0,0,1,0
4,5000,7.9,156.46,3,36000,1,852,11.2,0,734,…,0,0,1,1,0
5,7000,15.96,170.08,8,47004,1,280,23.51,0,694,…,0,0,0,0,1
6,3000,18.64,109.43,9,48000,1,900,5.35,0,664,…,0,0,0,1,0
7,5600,21.28,152.39,4,40000,0,958,5.55,0,679,…,1,0,0,0,1
8,5375,12.69,121.45,0,15000,0,774,18.08,0,729,…,0,0,0,0,1


# Summary Statistics

```
df.sum()
df.max()
df.min()
df.mean()
df.sd()
df.mode()
df.nmodal()
df.nunique()
```

In [45]:
df.mean()

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,zip_code,dti,delinq_2yrs,fico_range_high,…,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,11137.2,11.9828,323.311,4.82605,68889.1,0.855651,502.383,13.2951,0.146617,718.996,…,0.0457811,0.00963008,0.0239733,0.741262,0.258738


# Data Manipulation

## Select columns

In [46]:
# Select first 10 rows of the annual income column, `annual_inc`
df[:, 'annual_inc'].head()

,annual_inc
,▪▪▪▪▪▪▪▪
0,24000
1,30000
2,12252
3,49200
4,36000
5,47004
6,48000
7,40000
8,15000


In [56]:
# Select first 10 rows of annual income and loan status columns
df[:, "annual_inc", "loan_status"].head()

,loan_status,annual_inc
,▪,▪▪▪▪▪▪▪▪
0,0,30000
1,0,40000
2,0,15000
3,0,30000
4,0,100000
5,0,105000
6,0,76000
7,0,60000
8,0,50004


In [57]:
# Select all columns from loan_amnt to loan_status
df[:, 'loan_amnt':'loan_status']

# Alternative
# df[:, dt.f['loan_amnt':'loan_status']]

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪
0,5000,10.65,162.87,10,24000,1
1,2500,15.27,59.83,0,30000,0
2,2400,15.96,84.33,10,12252,1
3,10000,13.49,339.31,10,49200,1
4,5000,7.9,156.46,3,36000,1
5,7000,15.96,170.08,8,47004,1
6,3000,18.64,109.43,9,48000,1
7,5600,21.28,152.39,4,40000,0
8,5375,12.69,121.45,0,15000,0


In [79]:
# Select the last column
df[:, dt.f[-1]]

,term_ 60 months
,▪▪▪▪▪▪▪▪
0,0
1,1
2,0
3,0
4,0
5,1
6,0
7,1
8,1


In [80]:
# Select all integer columns
df[:, dt.f[int]]

# Select all float columns
df[:, dt.f[float]]

,emp_length
,▪▪▪▪
0,10
1,0
2,10
3,10
4,3
5,8
6,9
7,4
8,0


# Filtering Rows

In Python-flavored datatable, there is the concept of the **frame proxy** (`f`) which provides a simple way to refer to the `Frame` that we are currently operating upon.

In [63]:
# Filter rows where interest rate > 10% *AND* FICO < 650
df[ (dt.f.int_rate > 10) & (dt.f.fico_range_high < 650), :]

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,fico_range_high,inq_last_6mths,…,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,1500,14.7,51.78,1,25000,1,3.84,1,634,1,…,0,0,0,1,0
1,2700,15.96,94.88,1,52200,1,12.57,0,629,1,…,0,0,0,1,0


In [47]:
# Filter rows where loan status is `1`
df[dt.f.loan_status == 1, :]

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,zip_code,dti,delinq_2yrs,fico_range_high,…,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,5000,10.65,162.87,10,24000,1,860,27.65,0,739,…,0,0,0,1,0
1,2400,15.96,84.33,10,12252,1,606,8.72,0,739,…,1,0,0,1,0
2,10000,13.49,339.31,10,49200,1,917,20,0,694,…,0,0,0,1,0
3,5000,7.9,156.46,3,36000,1,852,11.2,0,734,…,0,0,1,1,0
4,7000,15.96,170.08,8,47004,1,280,23.51,0,694,…,0,0,0,0,1
5,3000,18.64,109.43,9,48000,1,900,5.35,0,664,…,0,0,0,1,0
6,6500,14.65,153.45,5,72000,1,853,16.12,0,699,…,0,0,0,0,1
7,12000,12.69,402.54,10,75000,1,913,10.78,0,679,…,0,0,0,1,0
8,3000,9.91,96.68,3,15000,1,606,12.56,0,709,…,0,0,0,1,0


# Sorting

In [48]:
# Sort the table by FICO score
df.sort('fico_range_high')

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,zip_code,dti,delinq_2yrs,fico_range_high,…,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,2700,15.96,94.88,1,52200,1,21,12.57,0,629,…,0,0,0,1,0
1,1500,14.7,51.78,1,25000,1,215,3.84,1,634,…,0,0,0,1,0
2,3000,18.64,109.43,9,48000,1,900,5.35,0,664,…,0,0,0,1,0
3,8000,16.77,284.31,0,62000,1,225,21.64,0,664,…,0,0,0,1,0
4,6400,16.77,227.45,5,75000,0,923,20.22,0,664,…,0,0,0,1,0
5,18000,19.91,475.99,10,65000,1,331,6.81,0,664,…,0,0,0,0,1
6,15000,18.64,547.12,1,54000,1,300,19.69,0,664,…,0,0,0,1,0
7,5300,15.27,184.43,1,30000,1,62,24.88,0,664,…,0,0,0,1,0
8,4500,15.96,158.12,5,65000,1,120,14.66,0,664,…,0,0,0,1,0


## Compute columnar expressions

In [71]:
df[:, {"new_int_rate": dt.f.int_rate - 5, "loan_over_inc": dt.f.loan_amnt / dt.f.annual_inc}]

,new_int_rate,loan_over_inc
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,5.65,0.208333
1,10.27,0.0833333
2,10.96,0.195886
3,8.49,0.203252
4,2.9,0.138889
5,10.96,0.148923
6,13.64,0.0625
7,16.28,0.14
8,7.69,0.358333


## Delete rows/columns

```python
del df[:, "D"]     # delete column D
del df[dt.f.A < 0, :] # delete rows where column A has negative values
```


In [49]:
# Delete the zip_code column

del df[:, 'zip_code']

df.head() # notice that 'zip_code' is gone!

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,fico_range_high,inq_last_6mths,…,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,5000,10.65,162.87,10,24000,1,27.65,0,739,1,…,0,0,0,1,0
1,2500,15.27,59.83,0,30000,0,1,0,744,5,…,0,0,0,0,1
2,2400,15.96,84.33,10,12252,1,8.72,0,739,2,…,1,0,0,1,0
3,10000,13.49,339.31,10,49200,1,20,0,694,1,…,0,0,0,1,0
4,5000,7.9,156.46,3,36000,1,11.2,0,734,3,…,0,0,1,1,0
5,7000,15.96,170.08,8,47004,1,23.51,0,694,1,…,0,0,0,0,1
6,3000,18.64,109.43,9,48000,1,5.35,0,664,2,…,0,0,0,1,0
7,5600,21.28,152.39,4,40000,0,5.55,0,679,2,…,1,0,0,0,1
8,5375,12.69,121.45,0,15000,0,18.08,0,729,0,…,0,0,0,0,1


## GroupBy calculations

In [53]:
# Group by example - average income by # of inquiries in last 6 mos.
df[:, dt.mean(dt.f.annual_inc), dt.by(dt.f.inq_last_6mths)]

,inq_last_6mths,annual_inc
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,67327.6
1,1,68773.3
2,2,70673.2
3,3,73449.5
4,4,80724.4
5,5,80231.3
6,6,83668.6
7,7,73460.9
8,8,97928.3


## Append rows/columns

```
df1.cbind(df2, df3)
df1.rbind(df4, force=True)
```